In [15]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

In [16]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

# 데이터 불러오기

In [17]:
df_1_3 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv")
df_3_6 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_3.csv")
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv")
df_end = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_EndPrice_IPO.csv")

In [18]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률'],axis =1)

In [19]:
df_end_1 = df_end.drop(['종가 대비 3개월 수익률','종가 대비 6개월 수익률','NullCount'],axis =1)
df_end_3 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 6개월 수익률','NullCount'],axis =1)
df_end_6 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 3개월 수익률','NullCount'],axis =1)

# 공통 전처리

In [20]:
def process(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
        df = df.drop(y_name,axis = 1)
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
        df = df.drop(y_name,axis = 1)
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1)

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
    
    ## y_data 분포 확인 및 데이터프레임 생성
    df_1 = train_list[0]['Cat'].value_counts().reindex([1,2,3,4])
    df_2 = test_list[0]['Cat'].value_counts().reindex([1,2,3,4])
    df_y = pd.concat([df_1,df_2])
    
    
    for i in range(1,len(train_list)):
        a = train_list[i]['Cat'].value_counts().reindex([1,2,3,4])
        b = test_list[i]['Cat'].value_counts().reindex([1,2,3,4])
    
        new_data = pd.concat([a,b] ,axis =0)
        df_y = pd.concat([df_y,new_data],axis =1)
        
    df_y.columns = list(range(0,32))
    df_y.index = ['Train_1','Train_2','Train_3','Train_4',\
              'Test_1','Test_2','Test_3','Test_4']
    df_y.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
    
    return df_y

In [21]:
df_0_1 = process(df_0_1,'공모가 대비 1개월 수익률')

In [22]:
df_0_3 = process(df_0_3,'공모가 대비 3개월 수익률')

In [23]:
df_0_6 = process(df_0_6,'공모가 대비 6개월 수익률')

In [24]:
df_1_3 = process(df_1_3,'1_3_수익률')

In [25]:
df_3_6 = process(df_3_6,'3_6_수익률')

In [12]:
df_3_6

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_22,model_set_23,model_set_24,model_set_25,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31
Train_1,40,35,36,32,26,23,22,21,18,17,...,35,35,39,38,39,41,42,42,42,40
Train_2,113,105,100,108,105,103,98,98,92,99,...,161,164,165,158,160,150,150,148,161,154
Train_3,9,9,10,12,10,9,13,16,16,16,...,19,20,20,21,25,23,23,24,25,26
Train_4,6,5,4,8,8,8,6,14,13,14,...,28,23,23,22,20,19,18,18,20,20
Test_1,1,1,0,1,2,3,6,9,9,8,...,10,15,12,12,10,5,3,3,1,0
Test_2,22,22,24,26,24,32,38,51,54,47,...,37,37,39,37,43,41,50,47,28,13
Test_3,3,3,8,10,10,11,5,4,5,4,...,11,7,8,7,5,8,10,9,7,3
Test_4,4,4,4,9,9,10,12,7,9,8,...,4,3,4,4,6,4,6,6,4,3


In [26]:
df_end_1 = process(df_end_1,'종가 대비 1개월 수익률')
df_end_3 = process(df_end_3,'종가 대비 3개월 수익률')
df_end_6 = process(df_end_6,'종가 대비 6개월 수익률')

In [27]:
df_0_1.to_csv("0_1_종속변수 분포.csv",encoding = 'euc-kr')
df_0_3.to_csv("0_3_종속변수 분포.csv",encoding = 'euc-kr')
df_0_6.to_csv("0_6_종속변수 분포.csv",encoding = 'euc-kr')
df_1_3.to_csv("1_3_종속변수 분포.csv",encoding = 'euc-kr')
df_3_6.to_csv("3_6_종속변수 분포.csv",encoding = 'euc-kr')

In [28]:
df_end_1.to_csv("end_1_종속변수 분포.csv",encoding = 'euc-kr')
df_end_3.to_csv("end_3_종속변수 분포.csv",encoding = 'euc-kr')
df_end_6.to_csv("end_6_종속변수 분포.csv",encoding = 'euc-kr')